In [ ]:
!pip install --upgrade google-cloud-bigquery

In [ ]:
import os
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print(f'Archivo {fn} con tamaño {len(uploaded[fn])}')
  with open('/content/' + fn, 'wb') as f:
    f.write(uploaded[fn])
  os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '/content/' + fn
  projectID = fn.rsplit("-", 1)[0]

Saving predictor-d2cb0-88eadb5daf62.json to predictor-d2cb0-88eadb5daf62.json
Archivo predictor-d2cb0-88eadb5daf62.json con tamaño 2314


In [ ]:
from google.cloud import bigquery

In [ ]:
# Datos del GCP
gcp = "predictor-d2cb0"
bq_datos = "predictor-d2cb0"

client = bigquery.Client(project=gcp)
dataset = client.dataset(dataset_id=bq_datos)

In [ ]:
def get(sql):
  query = client.query(sql)
  result = query.result()
  return result.to_dataframe()

In [ ]:
query = """select * from `predictor-d2cb0.diabetes.tabla`"""

In [ ]:
df = get(query)

In [ ]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Output
0,0,125,96,0,0,22.5,262.0,21,0
1,0,100,70,26,50,30.8,597.0,21,0
2,0,102,75,23,0,0.0,572.0,21,0
3,0,113,80,16,0,31.0,874.0,21,0
4,0,177,60,29,478,34.6,1072.0,21,1


# Modelo ML

In [ ]:
X = df.iloc[:,:8].values
y = df.iloc[:, 8].values

In [ ]:
X

array([[  0. , 125. ,  96. , ...,  22.5, 262. ,  21. ],
       [  0. , 100. ,  70. , ...,  30.8, 597. ,  21. ],
       [  0. , 102. ,  75. , ...,   0. , 572. ,  21. ],
       ...,
       [ 14. , 100. ,  78. , ...,  36.6, 412. ,  46. ],
       [ 15. , 136. ,  70. , ...,  37.1, 153. ,  43. ],
       [ 17. , 163. ,  72. , ...,  40.9, 817. ,  47. ]])

In [ ]:
y

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [ ]:
import tensorflow as tf

In [ ]:
# Arquitectura de la Red Neuronal
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(12, input_dim = 8, activation='relu'))
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
# Compilación
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=150, batch_size=16)

In [ ]:
# Evaluar modelo
_, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))

24/24 [==============================] - 0s 2ms/step - loss: 0.4719 - accuracy: 0.7721
Accuracy: 77.21


In [ ]:
predictions = model.predict(X[0:5, :]).round()

In [ ]:
for i in range(5):
  print(f'Entrada: {X[i]} Predicción: {predictions[i]} Real: {y[i]}')

Entrada: [  0.  125.   96.    0.    0.   22.5 262.   21. ] Predicción: [0.] Real: 0
Entrada: [  0.  100.   70.   26.   50.   30.8 597.   21. ] Predicción: [0.] Real: 0
Entrada: [  0. 102.  75.  23.   0.   0. 572.  21.] Predicción: [0.] Real: 0
Entrada: [  0. 113.  80.  16.   0.  31. 874.  21.] Predicción: [0.] Real: 0
Entrada: [   0.   177.    60.    29.   478.    34.6 1072.    21. ] Predicción: [1.] Real: 1


In [ ]:
model.save('/content/pred.h5')

In [ ]:
model_lite = tf.keras.models.load_model('/content/pred.h5')
convert = tf.lite.TFLiteConverter.from_keras_model(model_lite)
new_model = convert.convert()
open('modelo.tflite','wb').write(new_model)

In [ ]:
import firebase_admin
firebase_admin.initialize_app(options={'projectId':gcp, 'storageBucket':gcp + '.appspot.com'})

In [ ]:
from firebase_admin import ml

# Subir archivo
source = ml.TFLiteGCSModelSource.from_tflite_model_file('modelo.tflite')

In [ ]:
# Crear formato del modelo
model_format = ml.TFLiteFormat(model_source=source)
sdk_model_1 = ml.Model(display_name="my_model", model_format=model_format)


# Realizar llamada API
firebase_model_1 = ml.create_model(sdk_model_1)
print(firebase_model_1.as_dict())

# Publicar
model_id = firebase_model_1.model_id
firebase_model_1 = ml.publish_model(model_id)